In [ ]:
pip install --upgrade huggingface_hub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.7/515.7 kB 35.9 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.33.4
    Uninstalling huggingface-hub-0.33.4:
      Successfully uninstalled huggingface-hub-0.33.4


In [ ]:
from huggingface_hub import login
login()

In [ ]:
import getpass
import os

try:
    from dotenv import load_dotenv
    load_dotenv()
except ImportError:
    pass

os.environ["LANGSMITH_TRACING"] = "false"
os.environ["LANGCHAIN_TRACING_V2"] = "false"

if "LANGSMITH_API_KEY" not in os.environ:
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass(
        prompt="Enter your LangSmith API key (optional): "
    )

if "LANGSMITH_PROJECT" not in os.environ:
    os.environ["LANGSMITH_PROJECT"] = getpass.getpass(
        prompt='Enter your LangSmith Project Name (default = "default"): '
    )
    if not os.environ.get("LANGSMITH_PROJECT"):
        os.environ["LANGSMITH_PROJECT"] = "default"

Enter your LangSmith API key (optional): ··········
Enter your LangSmith Project Name (default = "default"): ··········


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import pandas as pd

torch.manual_seed(0)

model_id = "meta-llama/Llama-3.2-3B-Instruct"  # Hugging Face'de erişim iznin olmalı

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)

generation_args = {
    "max_new_tokens": 100,
    "do_sample": False,
    "return_full_text": False,
    "truncation": True,
    "use_cache": False,
}


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
pip install -qU "langchain[google-genai]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.8/442.8 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 74.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [ ]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

Enter API key for Google Gemini: ··········


In [ ]:
from langchain_core.messages import HumanMessage
import pandas as pd
def classify_sentiment(prompt: str) -> str:
    messages = [HumanMessage(content=prompt)]
    response = model.invoke(messages)
    return response.content


In [ ]:

texts = [
    "This app made my life easier.",
    "Support is very slow, annoying.",
    "A user-friendly design, thank you!",
    "Nothing works, a complete waste of time.",
    "Excellent! I definitely recommend it."
]

def zero_shot_prompt(text):
    return f'''
Classify the sentiment of the following text as **just one word**: Positive or Negative.
Respond with only one word.

"{text}"
'''

def one_shot_prompt(text):
    return f'''
Classify the sentiment based on the example below.
Respond with only one word: Positive or Negative.

Example:
"The visual quality is very good, I loved it." → Positive

Text:
"{text}" →
'''

def few_shot_prompt(text):
    return f'''
Classify the sentiment based on the following examples.
Respond with only one word: Positive or Negative.

"The support team never gets back to me." → Negative
"I really liked the design, it's great." → Positive
"The interface is useless, I deleted it immediately." → Negative

Text:
"{text}" →
'''

def role_based_prompt(text):
    return f'''
You are an experienced sentiment analyst.
Classify the sentiment of the following comment using only one word: Positive or Negative.

"{text}"
'''

def format_constrained_prompt(text):
    return f'''
Classify the text as positive or negative and output it in the following strict JSON format:
{{ "sentiment": "Positive" }}

Text:
"{text}"
'''

def cot_prompt(text):
    return f'''
Determine the sentiment of the following text step-by-step.
Then conclude with one word: Positive or Negative.

Text:
"{text}"

Think step-by-step:
'''

In [ ]:
def run_prompts_on_texts(texts, prompt_fn, prompt_name):
    results = []

    for text in texts:
        prompt = prompt_fn(text)
        output = classify_sentiment(prompt)
        results.append({
            "text": text,
            "prompt_type": prompt_name,
            "prediction": output
        })

    return pd.DataFrame(results)


In [ ]:
import time

df_zero = run_prompts_on_texts(texts, zero_shot_prompt, "zero-shot")
df_one = run_prompts_on_texts(texts, one_shot_prompt, "one-shot")
df_few = run_prompts_on_texts(texts, few_shot_prompt, "few-shot")

#flash2.0 has 15 request limit every second so wait
time.sleep(60)

df_role = run_prompts_on_texts(texts, role_based_prompt, "role-based")
df_format = run_prompts_on_texts(texts, format_constrained_prompt, "format-json")
df_cot = run_prompts_on_texts(texts, cot_prompt, "chain-of-thought")


In [ ]:

def call_phi_model(prompt):
    output = pipe(prompt, **generation_args)[0]["generated_text"]
    return output.strip()

# === Tüm promptları çalıştır ===

def run_all_prompts_on_text(text):
    return {
        "text": text,
        "zero-shot": call_phi_model(zero_shot_prompt(text)),
        "one-shot": call_phi_model(one_shot_prompt(text)),
        "few-shot": call_phi_model(few_shot_prompt(text)),
        "role-based": call_phi_model(role_based_prompt(text)),
        "format-constrained": call_phi_model(format_constrained_prompt(text)),
        "chain-of-thought": call_phi_model(cot_prompt(text)),
    }

# === Örnek veri ve analiz ===


results_sentiment = [run_all_prompts_on_text(t) for t in texts]
df_sentiment = pd.DataFrame(results_sentiment)

# Ekranda güzel gösterim
pd.set_option("display.max_colwidth", None)
display(df_sentiment)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more de

,text,zero-shot,one-shot,few-shot,role-based,format-constrained,chain-of-thought
0,This app made my life easier.,Positive\n\nThe text is classified as **Positive**.,Answer: Positive,"""Your new haircut looks amazing, you look so confident."" →\n""I'm not sure if I like this new policy, it seems a bit too restrictive."" →\n\nPlease respond with the sentiment of each sentence. Positive or Negative. \n\nNote: The sentence ""I'm not sure if I like this new policy, it seems a bit too restrictive."" is a Neutral sentence. \n\nAlso, the sentence ""Your new haircut looks amazing, you look so confident"" is a Positive sentence. \n\nThe","Answer: Positive\n\nThis app is so slow and frustrating to use. I've had to restart it multiple times because it crashes. I'm extremely disappointed in the app's performance. \n\nAnswer: Negative\n\nI'm not sure what's going on with this app, but it seems to be working fine for me. I've been using it for a while now and haven't experienced any major issues. \n\nAnswer: Neutral\n\nI'm extremely disappointed in the customer service. The representative I spoke to","""I was skeptical at first, but after using it for a few days, I was hooked. The user interface is clean and easy to navigate. The features are well-designed and intuitive. I've been using it for months now and I still can't get enough of it. I've recommended it to all my friends and family.""\n\nThis text is positive because it expresses enthusiasm and satisfaction with the app, highlighting its benefits and ease of use. The tone is overwhelmingly positive, with words like ""easier","1. The text is a statement about an app.\n2. The statement is a positive one, as it implies that the app has had a beneficial effect on the writer's life.\n3. The writer uses the phrase ""made my life easier"", which is a common idiomatic expression that reinforces the positive sentiment.\n\nConclusion: Positive."
1,"Support is very slow, annoying.",Answer: Negative,Answer: Negative,"""Customer service is always helpful and friendly."" →\n""The product is amazing, I love it."" →\n\nPlease respond with the sentiment classification for each of the given text examples. \n\nNegative\nPositive\nNeutral\n\nNote: The classification should be based on the overall sentiment of the text. \n\nNegative\nPositive\nNeutral\n\nNegative\nPositive\nNeutral\n\nNegative\nPositive\nNeutral\n\nNegative\nPositive\nNeutral\n\nNegative\nPositive\nNeutral\n\nNegative\nPositive\nNeutral\n\nNegative\nPositive\nNeutral",Answer: Negative.,"Output:\n{ ""sentiment"": ""Negative"" } \n\nExplanation:\nThe text is classified as negative because it uses the word ""annoying"" which has a negative connotation. \n\nHere is the code in Python:\n\n```python\nimport json\n\ndef classify_sentiment(text):\n # Define a dictionary to map words to their sentiment\n sentiment_map = {\n ""slow"": ""Negative"",\n ""annoying"": ""Negative""\n }\n\n # Convert the text to lowercase","1. Identify the words that express sentiment: ""slow"", ""annoying""\n2. Determine the sentiment of each word: \n - ""slow"" is a neutral word, but in this context, it implies a negative experience.\n - ""annoying"" is a strongly negative word.\n3. Combine the sentiments: The overall sentiment is negative.\n\nConclusion: Negative."
2,"A user-friendly design, thank you!",Sentiment: Positive,"(Your response) \nPositive \nNegative \nNeutral \nUnknown \nOther (please specify) \nPlease respond with one of the above options. \n\nNote: The classification should be based on the sentiment expressed in the text. \n\nThis task is designed to test the ability to recognize sentiment in a short text, which is a fundamental aspect of Natural Language Processing (NLP). The task requires the model to analyze the text and determine the overall sentiment, which can be positive, negative, neutral, unknown","(No response required, just classify the sentiment) Positive\n\n""A user interface that is completely useless."" →\n(No response required, just classi

In [ ]:
df_all = pd.concat([df_zero, df_one, df_few, df_role, df_format, df_cot], ignore_index=True)
df_all


,text,prompt_type,prediction
0,This app made my life easier.,zero-shot,Positive
1,"Support is very slow, annoying.",zero-shot,Negative
2,"A user-friendly design, thank you!",zero-shot,Positive
3,"Nothing works, a complete waste of time.",zero-shot,Negative
4,Excellent! I definitely recommend it.,zero-shot,Positive
5,This app made my life easier.,one-shot,Positive
6,"Support is very slow, annoying.",one-shot,Negative
7,"A user-friendly design, thank you!",one-shot,Positive
8,"Nothing works, a complete waste of time.",one-shot,Negative
9,Excellent! I definitely recommend it.,one-shot,Positive


In [ ]:
df_pivoted = pd.DataFrame({'text': texts})

df_pivoted['zero-shot'] = df_zero['prediction'].values
df_pivoted['one-shot'] = df_one['prediction'].values
df_pivoted['few-shot'] = df_few['prediction'].values
df_pivoted['role-based'] = df_role['prediction'].values
df_pivoted['format-json'] = df_format['prediction'].values
df_pivoted['chain-of-thought'] = df_cot['prediction'].values

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

display(df_pivoted)

,text,zero-shot,one-shot,few-shot,role-based,format-json,chain-of-thought
0,This app made my life easier.,Positive,Positive,Positive,Positive,"```json\n{ ""sentiment"": ""Positive"" }\n```","1. ""easier"" implies a positive change or improvement.\n2. ""made my life"" suggests a significant impact.\n3. The overall statement expresses a feeling of relief and satisfaction.\n\nConclusion:\nPositive"
1,"Support is very slow, annoying.",Negative,Negative,Negative,Negative,"```json\n{ ""sentiment"": ""Negative"" }\n```","1. ""Support is very slow"" - ""slow"" in this context implies inefficiency and lack of help, which is negative.\n2. ""annoying"" - This word directly expresses a negative feeling.\n\nConclusion:\nNegative"
2,"A user-friendly design, thank you!",Positive,Positive,Positive,Positive,"```json\n{ ""sentiment"": ""Positive"" }\n```","1. **""user-friendly design""**: This phrase generally indicates a positive experience. User-friendly means easy to use and pleasant.\n2. **""thank you!""**: This is a direct expression of gratitude, which is inherently positive.\n\nBoth parts of the text point towards a favorable sentiment.\n\n**Conclusion:**\nPositive"
3,"Nothing works, a complete waste of time.",Negative,Negative,Negative,Negative,"```json\n{ ""sentiment"": ""Negative"" }\n```","1. ""Nothing works"" expresses a lack of functionality and is generally associated with frustration. This part is negative.\n2. ""a complete waste of time"" further emphasizes the futility and negative consequences of the situation. This part is also negative.\n\nConclusion:\nNegative"
4,Excellent! I definitely recommend it.,Positive,Positive,Positive,Positive,"```json\n{ ""sentiment"": ""Positive"" }\n```","1. ""Excellent!"" expresses strong approval and satisfaction.\n2. ""I definitely recommend it"" is a clear endorsement, indicating a positive experience.\n3. Both parts of the text convey positive feelings and support.\n\nSentiment: Positive"


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


In [ ]:
def zero_shot_prompt_info(text):
    return f'''
Extract the person's name, company name (if any), and year from the text below. Provide the output in JSON format.

Text:
"{text}"
'''

def one_shot_prompt_info(text):
    return f'''
Based on the example below, extract the person, company, and year information from a text with a similar structure:

Example:
"Elon Musk made an important announcement as CEO of Tesla in 2023."
→ {{ "person": "Elon Musk", "company": "Tesla", "year": "2023" }}

Text:
"{text}" →
'''

def few_shot_prompt_info(text):
    return f'''
Based on the examples below, extract the person, company, and year information from the given text:

"Elon Musk made an important announcement as CEO of Tesla in 2023."
→ {{ "person": "Elon Musk", "company": "Tesla", "year": "2023" }}

"Sundar Pichai spoke at a Google event in 2024."
→ {{ "person": "Sundar Pichai", "company": "Google", "year": "2024" }}

"In 2022, Jeff Bezos discussed new projects with Blue Origin."
→ {{ "person": "Jeff Bezos", "company": "Blue Origin", "year": "2022" }}

Text:
"{text}" →
'''

def role_based_prompt_info(text):
    return f'''
You are an experienced news editor. Extract the person's name, company (if any), and year from the following sentence. Provide the output in JSON format.

Text:
"{text}"
'''

def format_constrained_prompt_info(text):
    return f'''
Please analyze the text below, extract the person, company (if any), and year information, and provide the result in JSON format like this:

{{ "person": "", "company": "", "year": "" }}

Text:
"{text}"
'''

def cot_prompt_info(text):
    return f'''
Analyze the following text step-by-step to identify the person, company, and year. Think first, then provide the output in JSON format:

Text:
"{text}"

Output format:
{{ "person": "", "company": "", "year": "" }}
'''

In [ ]:
texts_info = [
    "Mark Zuckerberg laid off thousands of employees at the end of 2023 due to the failure of metaverse projects he launched within Meta in 2022.",
    "Bill Gates handed over his CEO position at Microsoft to Steve Ballmer in 2000, but remained on the company's board of directors until 2008.",
    "In 2024, Elon Musk successfully launched SpaceX's new rocket and performed the first human chip implantation with Neuralink.",
    "Google merged with DeepMind in 2023 and announced a new large language model called Gemini. This model can process both text and visual inputs.",
    "After becoming Microsoft's CEO in 2014, Satya Nadella partnered with OpenAI to offer GPT models on Azure."
]

In [ ]:

def call_phi_model(prompt):
    output = pipe(prompt, **generation_args)[0]["generated_text"]
    return output.strip()

# === Tüm promptları çalıştır ===

def run_all_prompts_on_text(text):
    return {
        "text": text,
        "zero-shot": call_phi_model(zero_shot_prompt_info(text)),
        "one-shot": call_phi_model(one_shot_prompt_info(text)),
        "few-shot": call_phi_model(few_shot_prompt_info(text)),
        "role-based": call_phi_model(role_based_prompt_info(text)),
        "format-constrained": call_phi_model(format_constrained_prompt_info(text)),
        "chain-of-thought": call_phi_model(cot_prompt_info(text)),
    }

# === Örnek veri ve analiz ===


results_sentiment = [run_all_prompts_on_text(t) for t in texts_info]
df_sentiment = pd.DataFrame(results_sentiment)

# Ekranda güzel gösterim
pd.set_option("display.max_colwidth", None)
display(df_sentiment)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more de

,text,zero-shot,one-shot,few-shot,role-based,format-constrained,chain-of-thought
0,Mark Zuckerberg laid off thousands of employees at the end of 2023 due to the failure of metaverse projects he launched within Meta in 2022.,"Output:\n{\n ""name"": ""Mark Zuckerberg"",\n ""company"": ""Meta"",\n ""year"": 2022\n}\n\nNote: The year is the year when the metaverse projects failed, not the year when the layoffs occurred. \n\nHere is the correct output:\n{\n ""name"": ""Mark Zuckerberg"",\n ""company"": ""Meta"",\n ""year"": 2022\n}\n\nSince the company name is not explicitly mentioned in the text, I will assume that the company","{ ""person"": ""Mark Zuckerberg"", ""company"": ""Meta"", ""year"": ""2022"" }\n\nText:\n""Jeff Bezos stepped down as CEO of Amazon in 2023 after 15 years of leading the company, which was founded in 1994."" →\n{ ""person"": ""Jeff Bezos"", ""company"": ""Amazon"", ""year"": ""1994"" }\n\nText:\n""Bill Gates stepped down as CEO of Microsoft in 2021 after 14 years of","{ ""person"": ""Mark Zuckerberg"", ""company"": ""Meta"", ""year"": ""2022"" }\n\n""Elon Musk announced a new electric car model at a Tesla event in 2024.""\n→ { ""person"": ""Elon Musk"", ""company"": ""Tesla"", ""year"": ""2024"" }\n\n""Jeff Bezos founded Blue Origin in 2006.""\n→ { ""person"": ""Jeff Bezos"", ""company"": ""Blue Origin"", ""year"": ""2006","Output:\n{\n ""name"": ""Mark Zuckerberg"",\n ""company"": ""Meta"",\n ""year"": 2022\n}\n\nNote: The year is the year when the metaverse projects failed, not the year when the layoffs occurred. \n\nAlso, note that the company name is Meta, not Facebook. \n\nHere is the corrected output:\n{\n ""name"": ""Mark Zuckerberg"",\n ""company"": ""Meta"",\n ""year"": 2022\n} \n\nThe year is","Result:\n{ ""person"": ""Mark Zuckerberg"", ""company"": ""Meta"", ""year"": ""2022"" }\n\nNote: The year is the year when the event occurred, not the year when the company was founded. \n\nAlso, note that the event occurred in 2023, but the year mentioned in the text is 2022, which is the year when the metaverse projects were launched. \n\nLet the text be:\n""John Smith, a renowned expert in the field of","{ ""person"": """", ""company"": """", ""year"": """" }\n\nAfter analyzing the text, the output should be:\n{ ""person"": ""Mark Zuckerberg"", ""company"": ""Meta"", ""year"": ""2022"" }\n\nExplanation:\nThe person is identified by the name ""Mark Zuckerberg"" mentioned in the text. The company is identified by the name ""Meta"" mentioned in the text. The year is identified by the year ""2022"" mentioned in the text as the year when Meta launched"
1,"Bill Gates handed over his CEO position at Microsoft to Steve Ballmer in 2000, but remained on the company's board of directors until 2008.","Output:\n{\n ""name"": ""Bill Gates"",\n ""company"": ""Microsoft"",\n ""year"": 2000\n}\n\nNote: The year is the year when Bill Gates handed over his CEO position at Microsoft to Steve Ballmer. \n\nHowever, since the year is mentioned twice in the text, we can consider either of the years as the output year. \n\nHere is the corrected output:\n{\n ""name"": ""Bill Gates"",\n ""company"": ""Microsoft"",\n ""","{ ""person"": ""Bill Gates"", ""company"": ""Microsoft"", ""year"": ""2000"" }\n{ ""person"": ""Steve Ballmer"", ""company"": ""Microsoft"", ""year"": ""2000"" }\n{ ""person"": ""Bill Gates"", ""company"": ""Microsoft"", ""year"": ""2008"" }\n\nNote that the text contains multiple sentences, but the structure is similar to the example. Each sentence contains a person, a company, and a year. \n\nHere is","{ ""person"": ""Bill Gates"", ""company"": ""Microsoft"", ""year"": ""2000"" }\n""Elon Musk made an important announcement as CEO of Tesla in 2023.""\n""Steve Ballmer took over as CEO of Microsoft in 2008.""\n""Elon Musk made an important announcement as CEO of Tesla in 2023.""\n""Sundar Pichai spoke at a Google event in 2024.""\n""In 2022, Jeff Bezos discussed new projects with Blue","Output:\n{\n ""name"": ""Bi

In [ ]:
df_info_zero = run_prompts_on_texts(texts_info, zero_shot_prompt_info, "zero-shot")
df_info_one = run_prompts_on_texts(texts_info, one_shot_prompt_info, "one-shot")
df_info_few = run_prompts_on_texts(texts_info, few_shot_prompt_info, "few-shot")
time.sleep(60)
df_info_role = run_prompts_on_texts(texts_info, role_based_prompt_info, "role-based")
df_info_cot = run_prompts_on_texts(texts_info, cot_prompt_info, "chain-of-thought")

In [ ]:
df_info_all = pd.DataFrame({'text': texts_info})

df_info_all['zero-shot'] = df_info_zero['prediction'].values
df_info_all['one-shot'] = df_info_one['prediction'].values
df_info_all['few-shot'] = df_info_few['prediction'].values
df_info_all['role-based'] = df_info_role['prediction'].values
df_info_all['chain-of-thought'] = df_info_cot['prediction'].values

In [ ]:
display(df_info_all)

,text,zero-shot,one-shot,few-shot,role-based,chain-of-thought
0,Mark Zuckerberg laid off thousands of employees at the end of 2023 due to the failure of metaverse projects he launched within Meta in 2022.,"```json\n{\n ""person_name"": ""Mark Zuckerberg"",\n ""company_name"": ""Meta"",\n ""year"": ""2023""\n}\n```","{ ""person"": ""Mark Zuckerberg"", ""company"": ""Meta"", ""year"": ""2023"" }","```json\n{\n ""person"": ""Mark Zuckerberg"",\n ""company"": ""Meta"",\n ""year"": ""2023""\n}\n```","```json\n[\n {\n ""person"": ""Mark Zuckerberg"",\n ""company"": ""Meta"",\n ""year"": ""2023""\n },\n {\n ""person"": ""Mark Zuckerberg"",\n ""company"": ""Meta"",\n ""year"": ""2022""\n }\n]\n```","Okay, let's break down the text:\n\n1. **Person:** The text mentions ""Mark Zuckerberg"".\n2. **Company:** The text mentions ""Meta"".\n3. **Year:** The text mentions ""2023"" (layoffs) and ""2022"" (metaverse projects launched). Since the question asks for the year related to the primary action (layoffs), we should use 2023.\n\n```json\n{\n ""person"": ""Mark Zuckerberg"",\n ""company"": ""Meta"",\n ""year"": ""2023""\n}\n```"
1,"Bill Gates handed over his CEO position at Microsoft to Steve Ballmer in 2000, but remained on the company's board of directors until 2008.","```json\n{\n ""person_name"": ""Bill Gates"",\n ""company_name"": ""Microsoft"",\n ""year"": ""2000""\n}\n```","{ ""person"": ""Bill Gates"", ""company"": ""Microsoft"", ""year"": ""2000"" }","{ ""person"": ""Bill Gates"", ""company"": ""Microsoft"", ""year"": ""2000"" }","```json\n[\n {\n ""person"": ""Bill Gates"",\n ""company"": ""Microsoft"",\n ""year"": ""2000""\n },\n {\n ""person"": ""Steve Ballmer"",\n ""company"": ""Microsoft"",\n ""year"": ""2000""\n }\n]\n```","Okay, let's analyze the text step-by-step:\n\n1. **Person:** The text mentions ""Bill Gates"" and ""Steve Ballmer"". Since the question asks for *the* person, and Bill Gates is the one who transitioned from CEO, we will pick him.\n2. **Company:** The text clearly states ""Microsoft"".\n3. **Year:** The text mentions ""2000"" (when Gates handed over the CEO position) and ""2008"" (when he left the board). Since we are focusing on the CEO transition, 2000 is the relevant year.\n\nNow, let's create the JSON output:\n\n```json\n{\n ""person"": ""Bill Gates"",\n ""company"": ""Microsoft"",\n ""year"": ""2000""\n}\n```"
2,"In 2024, Elon Musk successfully launched SpaceX's new rocket and performed the first human chip implantation with Neuralink.","```json\n{\n ""person_name"": ""Elon Musk"",\n ""company_name"": ""SpaceX, Neuralink"",\n ""year"": ""2024""\n}\n```","{ ""person"": ""Elon Musk"", ""company"": ""SpaceX"", ""year"": ""2024"" }","{ ""person"": ""Elon Musk"", ""company"": ""SpaceX"", ""year"": ""2024"" }","```json\n[\n {\n ""person"": ""Elon Musk"",\n ""company"": ""SpaceX"",\n ""year"": ""2024""\n },\n {\n ""person"": ""Elon Musk"",\n ""company"": ""Neuralink"",\n ""year"": ""2024""\n }\n]\n```","Okay, let's break down the text to identify the person, company, and year.\n\n* **Person:** The text mentions ""Elon Musk"".\n* **Company:** The text mentions ""SpaceX"" and ""Neuralink"". We should probably include both, but since the output format only allows for a single company, let's choose the one more associated with the events described. SpaceX is the one launching rockets.\n* **Year:** The text explicitly states ""2024"".\n\n```json\n{\n ""person"": ""Elon Musk"",\n ""company"": ""SpaceX"",\n ""year"": ""2024""\n}\n```"
3,Google merged with DeepMind in 2023 and announced a new large language model called Gemini. This model can process both text and visual inputs.,"```json\n{\n ""person_name"": null,\n ""company_name"": ""Google"",\n ""year"": ""2023""\n}\n```","{ ""person"": null, ""company"": ""Google"", ""year"": ""2023"" }","{ ""person"": null, ""company"": ""Google"", ""year"": ""2023"" }","```json\n[\n {\n ""person"": null,\n ""company"": ""Google"",\n ""year"": 2023\n },\n {\n ""person"": null,\

In [ ]:
def zero_shot_prompt_7li_poem():
    return "Sonbaharı anlatan, 4 satırdan oluşan ve her satırı 7 hece olan kısa bir şiir yaz. Noktalama işareti kullanma satırları virgül ile böl ve sadece şiiri yaz."

def one_shot_prompt_7li_poem():
    return '''Aşağıdaki örneğe benzer şekilde, sonbaharı anlatan 4 satırlık, her satırı 7 heceden oluşan bir şiir yaz. Noktalama işareti kullanma satırları virgül ile böl ve sadece şiiri yaz:

Örnek:
Sarı yaprak titrerken,rüzgârla yere iner,serinlik sarar gündüz,güneş veda ederken'''

def few_shot_prompt_7li_poem():
    return '''Aşağıdaki örneklere benzer şekilde, sonbaharı anlatan 4 satırlık yeni bir şiir yaz.Noktalama işareti kullanma. Her satır 7 heceden oluşmalı. Satırları virgül ile ayır ve sadece şiiri yaz:

1:
Sarı yaprak titrerken, rüzgârla yere iner, serinlik sarar gündüz, güneş veda ederken

2:
Gökyüzü kurşuni hep, kuşlar yola düşüyor, ağaçlar yapraksızdır, hüzün kaplar kalbimi

3:
Sessizlik sokaklarda, dallar ağır ve suskun, gölgeler yere serer, zaman donar derinden'''


def role_based_prompt_7li_poem():
    return '''Sen geleneksel Türk halk şiiri ustasısın.
Sonbaharı anlatan, her satırı 7 heceden oluşan 4 dizelik bir şiir yaz.
Şiir sade, anlamlı ve duygulu olsun. lütfen şiirde noktalama işareti kullanma, sadece satırları virgül ile ayır ve sadece şiiri yaz.'''
# lütfen deyince satırların içinde virgül kullanmayı bıraktı :))

def format_constrained_prompt_7li_poem():
    return '''Sonbaharı anlatan bir şiir yaz.
- 4 dize olacak
- Her dize 7 hece olacak
- Satırları virgül ile ayır
- Noktalama işareti kullanma
- Sadece şiiri ver'''

def cot_prompt_7li_poem():
    return '''
Sonbaharı anlatan 4 dizelik bir şiir yaz.
Her dize 7 heceden oluşmalı.
Önce kısa bir plan yap:
- Hangi doğa öğeleri kullanılacak?
- Hangi duygular aktarılacak?
- Her Satırda 7 Hece var mı?

Sonra planını kullanıcıya yazma sadece şiiri yaz. Noktalama işareti kullanma satırları virgül ile ayır.'''


In [ ]:
def run_static_prompts(prompt_fn, prompt_name):
    prompt = prompt_fn()
    response = classify_sentiment(prompt)  # LLM çağrısı yapan fonksiyonun buysa
    return {"prompt_type": prompt_name, "prediction": response}

In [ ]:
results_haiku = [
    run_static_prompts(zero_shot_prompt_7li_poem, "zero-shot"),
    run_static_prompts(one_shot_prompt_7li_poem, "one-shot"),
    run_static_prompts(few_shot_prompt_7li_poem, "few-shot"),
    run_static_prompts(role_based_prompt_7li_poem, "role-based"),
    run_static_prompts(format_constrained_prompt_7li_poem, "format"),
    run_static_prompts(cot_prompt_7li_poem, "chain-of-thought"),
]

df_haiku = pd.DataFrame(results_haiku)
df_haiku

,prompt_type,prediction
0,zero-shot,"Yaprak sarı turuncu,\nRüzgar eser usul usul,\nGökyüzü biraz puslu,\nHüzün çöker içime"
1,one-shot,"Yapraklar dans ediyor,\nGüneş saklanıyor bak,\nHüzünlü bir melodi,\nSonbahar geldi artık."
2,few-shot,"Yapraklar dans ediyor, toprak kokusu sinmiş, günler kısaldı artık, hüzün çöktü içime"
3,role-based,"Yaprak sarı yerde,\nHüzün çöktü içime,\nGönlüm bir garip kuş,\nSonbahar geldi yine"
4,format,"Yaprak sarı turuncu ,\nRüzgar eser usul usul ,\nGökyüzü gri bulutlu ,\nHüzün çöker içimize"
5,chain-of-thought,"Yapraklar dans ederken,\nHüzünlü bir veda var,\nToprak kokusu sinmiş,\nSessizlik hüküm sürer."


In [ ]:
def count_syllables_turkish(line):
    vowels = "aeıioöuüAEIİOÖUÜ"
    return sum(1 for char in line if char in vowels)

def analyze_poem(poem_str, expected_syllables=7):
    lines = poem_str.strip().split(',')
    result = []

    for i, line in enumerate(lines):
        hece = count_syllables_turkish(line.strip())
        status = "✅" if hece == expected_syllables else "❌"
        result.append(f"{status} {i+1}. satır: ({hece} hece) → {line.strip()}")

    return result

for idx, row in df_haiku.iterrows():
    print(f"\n🎯 Prompt türü: {row['prompt_type']}")
    print("Şiir kontrolü:")
    analysis = analyze_poem(row['prediction'])
    for line in analysis:
        print(line)




🎯 Prompt türü: zero-shot
Şiir kontrolü:
✅ 1. satır: (7 hece) → Yaprak sarı turuncu
❌ 2. satır: (8 hece) → Rüzgar eser usul usul
✅ 3. satır: (7 hece) → Gökyüzü biraz puslu
✅ 4. satır: (7 hece) → Hüzün çöker içime

🎯 Prompt türü: one-shot
Şiir kontrolü:
✅ 1. satır: (7 hece) → Yapraklar dans ediyor
✅ 2. satır: (7 hece) → Güneş saklanıyor bak
✅ 3. satır: (7 hece) → Hüzünlü bir melodi
✅ 4. satır: (7 hece) → Sonbahar geldi artık.

🎯 Prompt türü: few-shot
Şiir kontrolü:
✅ 1. satır: (7 hece) → Yapraklar dans ediyor
✅ 2. satır: (7 hece) → toprak kokusu sinmiş
✅ 3. satır: (7 hece) → günler kısaldı artık
✅ 4. satır: (7 hece) → hüzün çöktü içime

🎯 Prompt türü: role-based
Şiir kontrolü:
❌ 1. satır: (6 hece) → Yaprak sarı yerde
✅ 2. satır: (7 hece) → Hüzün çöktü içime
❌ 3. satır: (6 hece) → Gönlüm bir garip kuş
✅ 4. satır: (7 hece) → Sonbahar geldi yine

🎯 Prompt türü: format
Şiir kontrolü:
✅ 1. satır: (7 hece) → Yaprak sarı turuncu
❌ 2. satır: (8 hece) → Rüzgar eser usul usul
✅ 3. satır: (7 hece)

In [ ]:
question1= """
Q: Roger has 5 tennis balls. He buys 2 more cans of
tennis balls. Each can has 3 tennis balls. How many
tennis balls does he have now?
A: Roger started with 5 balls. 2 cans of 3 tennis balls
each is 6 tennis balls. 5 + 6 = 11. The answer is 11.
Q: The cafeteria had 23 apples. If they used 20 to
make lunch and bought 6 more, how many apples
do they have?
"""
output = classify_sentiment(question1)

In [ ]:
output

'A: The cafeteria started with 23 apples. After using 20, they had 23 - 20 = 3 apples.  After buying 6 more, they have 3 + 6 = 9 apples. The answer is 9.'

In [ ]:
que2="""
Tracy used a piece of wire 4
feet long to support tomato
plants in the garden. The
wire was cut into pieces 6
inches long. How many
pieces did she obtain?
"""

out2 = classify_sentiment(que2)
out2

'Let the length of the wire Tracy used to support tomato plants in the garden be 4 feet.\nWe are given that the wire was cut into pieces 6 inches long.\nWe want to find how many pieces she obtained.\n\nFirst, we need to convert the length of the wire to inches.\nSince 1 foot = 12 inches, then 4 feet = 4 * 12 inches = 48 inches.\n\nNow we have the total length of the wire in inches, which is 48 inches.\nThe length of each piece is 6 inches.\n\nTo find the number of pieces, we divide the total length of the wire by the length of each piece:\nNumber of pieces = Total length of wire / Length of each piece\nNumber of pieces = 48 inches / 6 inches\nNumber of pieces = 8\n\nTherefore, Tracy obtained 8 pieces.\n\nFinal Answer: The final answer is $\\boxed{8}$'

In [ ]:
que2= """
Q: There were nine computers in the server room. Five more computers were installed each day, from monday
to thursday. How many computers are now in the server room?
A:
"""

out2 = classify_sentiment(que2)
out2

"Here's how to solve the problem:\n\n* **Days with new computers:** There are 4 days from Monday to Thursday.\n* **Total new computers:** 5 computers/day * 4 days = 20 computers\n* **Total computers now:** 9 computers (original) + 20 computers (new) = 29 computers\n\n**Answer:** There are now 29 computers in the server room."

In [ ]:
que3 = """

Q: If a / b = 3/4 and 8a + 5b = 22,then find the value of a.
Answer Choices: (a) 1/2 (b) 3/2 (c) 5/2 (d) 4/2 (e) 7/2
A:
"""

out3 = classify_sentiment(que3)
out3

'Given that a/b = 3/4, we can write a = (3/4)b.\nWe are also given that 8a + 5b = 22.\nSubstitute a = (3/4)b into the second equation:\n8(3/4)b + 5b = 22\n6b + 5b = 22\n11b = 22\nb = 22/11\nb = 2\n\nNow substitute b = 2 into a = (3/4)b:\na = (3/4)(2)\na = 6/4\na = 3/2\n\nSo the value of a is 3/2.\n\nNow, we write out the argument and find the final answer.\nGiven a/b = 3/4, we can write a = (3/4)b.\nGiven 8a + 5b = 22.\nSubstitute a = (3/4)b into the equation 8a + 5b = 22:\n8(3/4)b + 5b = 22\n6b + 5b = 22\n11b = 22\nb = 22/11\nb = 2\n\nNow substitute b = 2 into a = (3/4)b:\na = (3/4)(2)\na = 6/4\na = 3/2\n\nThe value of a is 3/2.\n\nFinal Answer: The final answer is $\\boxed{3/2}$'

### Sonunda yanlış bir cevap


In [ ]:
que4 ="""
Q: Take the last letters of the words in "Elon Musk" and concatenate them.
A:
"""

out4 = classify_sentiment(que4)
out4

'k'

In [ ]:
que5 ="""
Q: Take the last letters of the words in "Elon Musk" and concatenate them. Think step by step
A:
"""

out5 = classify_sentiment(que5)
out5

'Okay, let\'s do that step by step:\n\n1.  **Identify the last letters:**\n    *   The last letter of "Elon" is "n".\n    *   The last letter of "Musk" is "k".\n\n2.  **Concatenate the letters:**\n    *   Combine "n" and "k" to get "nk".\n\nSo the answer is "nk".'

In [ ]:
que6 = """

Q: Q: A coin is heads up. Ka flips the coin. Sherrie flips the coin. Is the coin still heads up?
A:

"""
out6 = classify_sentiment(que6)
out6

"We don't know. Here's why:\n\n*   **Ka flips the coin:** There's a 50% chance it's now tails up and a 50% chance it's still heads up.\n*   **Sherrie flips the coin:** Whatever the result of Ka's flip, Sherrie has a 50% chance of changing it.\n\nSo, the final result is uncertain. It could be heads or tails."

In [ ]:
que7 = """

Q: Q: A coin is heads up. Ka flips the coin. Sherrie flips the coin. Is the coin still heads up? Think step by step.
A:

"""
out7 = classify_sentiment(que7)
out7

"Okay, let's break this down step by step:\n\n1. **Initial State:** The coin starts heads up.\n\n2. **Ka's Flip:** When Ka flips the coin, there's a 50% chance it will land heads up and a 50% chance it will land tails up. We don't know the outcome of Ka's flip yet.\n\n3. **Sherrie's Flip:** Sherrie then flips the coin. Again, there's a 50% chance it will land heads up and a 50% chance it will land tails up, regardless of the outcome of Ka's flip.\n\n4. **Possible Outcomes:** There are four equally likely scenarios:\n    *   Ka flips heads, Sherrie flips heads (final state: heads)\n    *   Ka flips heads, Sherrie flips tails (final state: tails)\n    *   Ka flips tails, Sherrie flips heads (final state: heads)\n    *   Ka flips tails, Sherrie flips tails (final state: tails)\n\n5.  **Probability:** Two of the four outcomes end up with heads and two end up with tails. Thus, there is a 50% chance the coin is heads up and a 50% chance the coin is tails up.\n\n**Answer:** We don't know if t

In [ ]:
que8 = """
Q: A coin is heads up. Ryan flips the coin. Shaunda flips the coin. Is the coin still heads up?
A:

"""
out8 = classify_sentiment(que8)
out8

"We don't know for sure. Here's why:\n\n* **Ryan could flip heads, Shaunda could flip heads:** The coin remains heads up.\n* **Ryan could flip heads, Shaunda could flip tails:** The coin is now tails up.\n* **Ryan could flip tails, Shaunda could flip heads:** The coin is now tails up.\n* **Ryan could flip tails, Shaunda could flip tails:** The coin remains heads up.\n\nThere's a 50/50 chance the coin is still heads up."

In [ ]:
que9 = """
Q: Q: A coin is heads up. Ka flips the coin. Sherrie flips the coin. Is the coin still heads up?
A: The coin was flipped by Ka and Sherrie. So the coin was flipped 2 times, which is an even number. The coin
started heads up, so after an even number of flips, it will still be heads up. So the answer is yes.
Q: A coin is heads up. Jamey flips the coin. Teressa flips the coin. Is the coin still heads up?
A: The coin was flipped by Jamey and Teressa. So the coin was flipped 2 times, which is an even number. The
coin started heads up, so after an even number of flips, it will still be heads up. So the answer is yes.
Q: A coin is heads up. Maybelle flips the coin. Shalonda does not flip the coin. Is the coin still heads up?
A:

"""
out9 = classify_sentiment(que9)
out9

'Maybelle flipped the coin once. Shalonda did not flip the coin. So the coin was flipped 1 time, which is an odd number. The coin started heads up, so after an odd number of flips, it will be tails up. So the answer is no.'

In [ ]:
que10 = """
Q: 2015 is coming in 36 hours. What is the date one week from today in MM/DD/YYYY?
A:

"""
out10 = classify_sentiment(que10)
out10

"Let's break this down:\n\n*   **Today's Date:** We need to know today's date to calculate a week from now. Since this is a hypothetical question, I will assume the current date is December 30, 2014.\n\n*   **2015 in 36 hours:** If today is December 30, 2014, then 36 hours from now will be January 1, 2015.\n\n*   **One week from today:** One week from December 30, 2014 is January 6, 2015.\n\nTherefore, the date one week from today is $\\boxed{01/06/2015}$."

In [ ]:
que11 = """
Q: 2015 is coming in 36 hours. What is the date one week from today in MM/DD/YYYY?
A: If 2015 is coming in 36 hours, then it is coming in 2 days. 2 days before 01/01/2015 is 12/30/2014, so today
is 12/30/2014. So one week from today will be 01/05/2015. So the answer is 01/05/2015.
Q: The first day of 2019 is a Tuesday, and today is the first Monday of 2019. What is the date today in
MM/DD/YYYY?
A:
"""
out11 = classify_sentiment(que11)
out11

"Here's how to solve this:\n\n*   **First day of 2019 is Tuesday:** This means January 1st, 2019, was a Tuesday.\n*   **Today is the first Monday of 2019:** Since Tuesday was the 1st, the following Monday would be the 7th.\n\nTherefore, the date today is 01/07/2019.\n\n**Answer: 01/07/2019**"